In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install numpy scikit-learn
!pip install -q torch_geometric
!pip install -q class_resolver
!pip3 install pymatting
!pip install opencv-python
!pip install xgboost imbalanced-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 6.3 MB/s eta 0:00:00


In [ ]:
!pip install xgboost imbalanced-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 MB 23.9 MB/s  0:00:04
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [imbalanced-learn]


In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, log_loss
from sklearn.model_selection import StratifiedShuffleSplit
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_curve, roc_auc_score, log_loss
)
import torch
import torch.nn as nn
import torch.nn.functional as nnFn
import torch.optim as optim
import numpy as np
import random
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import TensorDataset, DataLoader, Subset
from torch_geometric.data import Data
from torch.utils.data import TensorDataset, DataLoader, Subset
from torchvision import models

In [ ]:
import torch
print("CUDA available:", torch.cuda.is_available())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

CUDA available: True
GPU Name: NVIDIA RTX A4000


In [ ]:
data = np.load('/home/snu/Downloads/pneumoniamnist_224.npz', allow_pickle=True)
all_images = np.concatenate([data['train_images'], data['val_images'], data['test_images']], axis=0)
all_labels = np.concatenate([data['train_labels'], data['val_labels'], data['test_labels']], axis=0).squeeze()

# Convert to 3-channel RGB
images = all_images.astype(np.float32) / 255.0
images = np.repeat(images[:, None, :, :], 3, axis=1)  # (N, 3, 224, 224)
X_torch = torch.tensor(images)
y_torch = torch.tensor(all_labels).long()

print(f"Raw images: {X_torch.shape}, Labels: {y_torch.shape}")

Raw images: torch.Size([5856, 3, 224, 224]), Labels: torch.Size([5856])


In [ ]:
class0_idx = [i for i in range(len(y_torch)) if y_torch[i] == 0]
class1_idx = [i for i in range(len(y_torch)) if y_torch[i] == 1]

random.seed(42)
sampled_class0 = random.sample(class0_idx, min(2000, len(class0_idx)))
sampled_class1 = random.sample(class1_idx, min(2000, len(class1_idx)))

selected_indices = sampled_class0 + sampled_class1
random.shuffle(selected_indices)

subset_dataset = Subset(TensorDataset(X_torch, y_torch), selected_indices)
subset_loader = DataLoader(subset_dataset, batch_size=64, shuffle=False)

In [ ]:
resnet = models.resnet18(pretrained=True)
resnet.fc = nn.Identity()
resnet = resnet.to(device)
resnet.eval()

resnet_feats = []
y_list = []
with torch.no_grad():
    for imgs, labels in subset_loader:
        imgs = imgs.to(device)
        feats = resnet(imgs)
        resnet_feats.append(feats.cpu())
        y_list.extend(labels.cpu().tolist())

features = torch.cat(resnet_feats, dim=0).numpy().astype(np.float32)  # shape (N, feat_dim)
y_labels = np.array(y_list).astype(np.float32)
print("Feature shape:", features.shape, "Label shape:", y_labels.shape)

/home/snu/anaconda3/envs/torch_env/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/snu/anaconda3/envs/torch_env/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Feature shape: (3583, 512) Label shape: (3583,)


In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(features)

In [ ]:
# Stratified split
sss = StratifiedShuffleSplit(n_splits=20, test_size=0.9, random_state=42)

In [ ]:
accuracies, precisions, recalls, f1_scores, losses, all_auc = [], [], [], [], [], []
all_y_true, all_y_proba, all_fpr, all_tpr = [], [], [], []

for fold, (train_idx, test_idx) in enumerate(sss.split(X, y_labels)):
    print(f"\n===== Fold {fold + 1} =====")

    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y_labels[train_idx], y_labels[test_idx]

    # Train XGBoost
    xgb = XGBClassifier(
        n_estimators=200,
        max_depth=6,
        learning_rate=0.1,
        eval_metric='logloss',
        random_state=42
    )
    xgb.fit(X_train, y_train)

    # Predictions
    y_pred = xgb.predict(X_test)
    y_pred_proba = xgb.predict_proba(X_test)[:, 1]

    # Metrics
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    auc = roc_auc_score(y_test, y_pred_proba)
    loss = log_loss(y_test, y_pred_proba)

    # Save fold metrics
    accuracies.append(acc)
    precisions.append(prec)
    recalls.append(rec)
    f1_scores.append(f1)
    losses.append(loss)
    all_auc.append(auc)

    # Save ROC curve data (optional)
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    all_fpr.append(fpr)
    all_tpr.append(tpr)
    all_y_true.extend(y_test)
    all_y_proba.extend(y_pred_proba)

    print(f"Accuracy: {acc:.4f} | Precision: {prec:.4f} | Recall: {rec:.4f} | "
          f"F1: {f1:.4f} | AUC: {auc:.4f} | Loss: {loss:.4f}")

# ==============================
# Final averaged results
# ==============================
print("\n===== Average Results Across 20 Folds =====")
print(f"Accuracy:  {np.mean(accuracies):.4f} \u00b1 {np.std(accuracies):.4f}")
print(f"Precision: {np.mean(precisions):.4f} \u00b1 {np.std(precisions):.4f}")
print(f"Recall:    {np.mean(recalls):.4f} \u00b1 {np.std(recalls):.4f}")
print(f"F1 Score:  {np.mean(f1_scores):.4f} \u00b1 {np.std(f1_scores):.4f}")
print(f"AUC:       {np.mean(all_auc):.4f} \u00b1 {np.std(all_auc):.4f}")
print(f"Log Loss:  {np.mean(losses):.4f} \u00b1 {np.std(losses):.4f}")


===== Fold 1 =====
Accuracy: 0.9243 | Precision: 0.9476 | Recall: 0.9150 | F1: 0.9310 | AUC: 0.9799 | Loss: 0.2048

===== Fold 2 =====
Accuracy: 0.9284 | Precision: 0.9188 | Recall: 0.9561 | F1: 0.9371 | AUC: 0.9797 | Loss: 0.2111

===== Fold 3 =====
Accuracy: 0.9274 | Precision: 0.9444 | Recall: 0.9244 | F1: 0.9343 | AUC: 0.9801 | Loss: 0.1922

===== Fold 4 =====
Accuracy: 0.9169 | Precision: 0.9227 | Recall: 0.9289 | F1: 0.9258 | AUC: 0.9748 | Loss: 0.2356

===== Fold 5 =====
Accuracy: 0.9225 | Precision: 0.9330 | Recall: 0.9278 | F1: 0.9304 | AUC: 0.9767 | Loss: 0.2134

===== Fold 6 =====
Accuracy: 0.9374 | Precision: 0.9529 | Recall: 0.9339 | F1: 0.9433 | AUC: 0.9827 | Loss: 0.1730

===== Fold 7 =====
Accuracy: 0.9278 | Precision: 0.9312 | Recall: 0.9400 | F1: 0.9356 | AUC: 0.9798 | Loss: 0.1952

===== Fold 8 =====
Accuracy: 0.9163 | Precision: 0.9069 | Recall: 0.9472 | F1: 0.9266 | AUC: 0.9790 | Loss: 0.2233

===== Fold 9 =====
Accuracy: 0.9225 | Precision: 0.9306 | Recall: 0.930

In [ ]:
accuracies, precisions, recalls, f1_scores, losses, all_auc = [], [], [], [], [], []
all_y_true, all_y_proba, all_fpr, all_tpr = [], [], [], []

for fold, (train_idx, test_idx) in enumerate(sss.split(X, y_labels)):
    print(f"\n===== Fold {fold + 1} =====")

    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y_labels[train_idx], y_labels[test_idx]

    # Train XGBoost
    xgb = XGBClassifier(
        n_estimators=200,
        max_depth=6,
        learning_rate=0.1,
        eval_metric='logloss',
        random_state=42
    )
    xgb.fit(X_train, y_train)

    # Predictions
    y_pred = xgb.predict(X_test)
    y_pred_proba = xgb.predict_proba(X_test)[:, 1]

    # Metrics
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    auc = roc_auc_score(y_test, y_pred_proba)
    loss = log_loss(y_test, y_pred_proba)

    # Save fold metrics
    accuracies.append(acc)
    precisions.append(prec)
    recalls.append(rec)
    f1_scores.append(f1)
    losses.append(loss)
    all_auc.append(auc)

    # Save ROC curve data (optional)
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    all_fpr.append(fpr)
    all_tpr.append(tpr)
    all_y_true.extend(y_test)
    all_y_proba.extend(y_pred_proba)

    print(f"Accuracy: {acc:.4f} | Precision: {prec:.4f} | Recall: {rec:.4f} | "
          f"F1: {f1:.4f} | AUC: {auc:.4f} | Loss: {loss:.4f}")

# ==============================
# Final averaged results
# ==============================
print("\n===== Average Results Across 20 Folds =====")
print(f"Accuracy:  {np.mean(accuracies):.4f} \u00b1 {np.std(accuracies):.4f}")
print(f"Precision: {np.mean(precisions):.4f} \u00b1 {np.std(precisions):.4f}")
print(f"Recall:    {np.mean(recalls):.4f} \u00b1 {np.std(recalls):.4f}")
print(f"F1 Score:  {np.mean(f1_scores):.4f} \u00b1 {np.std(f1_scores):.4f}")
print(f"AUC:       {np.mean(all_auc):.4f} \u00b1 {np.std(all_auc):.4f}")
print(f"Log Loss:  {np.mean(losses):.4f} \u00b1 {np.std(losses):.4f}")


===== Fold 1 =====
Accuracy: 0.8091 | Precision: 0.8239 | Recall: 0.9396 | F1: 0.8780 | AUC: 0.8145 | Loss: 0.4914

===== Fold 2 =====
Accuracy: 0.8276 | Precision: 0.8289 | Recall: 0.9630 | F1: 0.8909 | AUC: 0.8289 | Loss: 0.4568

===== Fold 3 =====
Accuracy: 0.7920 | Precision: 0.8137 | Recall: 0.9279 | F1: 0.8670 | AUC: 0.8204 | Loss: 0.4861

===== Fold 4 =====
Accuracy: 0.7963 | Precision: 0.8104 | Recall: 0.9415 | F1: 0.8711 | AUC: 0.7988 | Loss: 0.5165

===== Fold 5 =====
Accuracy: 0.8248 | Precision: 0.8397 | Recall: 0.9396 | F1: 0.8868 | AUC: 0.8057 | Loss: 0.4797

===== Fold 6 =====
Accuracy: 0.8105 | Precision: 0.8177 | Recall: 0.9532 | F1: 0.8803 | AUC: 0.8149 | Loss: 0.4749

===== Fold 7 =====
Accuracy: 0.7621 | Precision: 0.8168 | Recall: 0.8694 | F1: 0.8423 | AUC: 0.7953 | Loss: 0.5278

===== Fold 8 =====
Accuracy: 0.8120 | Precision: 0.8302 | Recall: 0.9337 | F1: 0.8789 | AUC: 0.8501 | Loss: 0.4256

===== Fold 9 =====
Accuracy: 0.8063 | Precision: 0.8095 | Recall: 0.961